In [1]:
import os
os.environ["NUMBA_DISABLE_JIT"] = "1"

import pandas as pd
import numpy as np
import vectorbt as vbt
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler

# === 1. Cargar df_test ===
df_test = pd.read_csv("df_test.csv").reset_index(drop=True)

# Agregar columnas temporales si existe 'time'
if 'time' in df_test.columns:
    df_test['time'] = pd.to_datetime(df_test['time'])
    df_test['hour'] = df_test['time'].dt.hour
    df_test['weekday'] = df_test['time'].dt.weekday

# === 2. Cargar modelo Keras ===
model_path = "best_model_walkforward_keras_optuna.h5"
model = load_model(model_path)
print(f"✅ Modelo cargado: {model_path}")

# === 3. Preparar features y escalar ===
features_cols = [
    'close', 'sma_200_vs_100', 'sma_50_vs_25', 'rsi_14', 'acceleration_12',
    'macd', 'stoch_k', 'momentum_12', 'bb_middle', 'bb_upper', 'bb_lower',
    'weighted_close', 'poc_12', 'high_channel_20', 'low_channel_20',
    'abs_diff_mrb_vs_close', 'last_signal_price_1', 'last_signal_price_2',
    'last_signal_price_3', 'last_signal_price_4', 'last_signal_price_5',
    'hour', 'weekday'
]
features_cols = [col for col in features_cols if col in df_test.columns]
X_test = df_test[features_cols].fillna(0).astype(float)

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_test)

# === 4. Predicción binaria y probabilidades ===
df_test['proba_1'] = model.predict(X_scaled).flatten()
df_test['pred'] = (df_test['proba_1'] > 0.5).astype(int)

# === 5. Señales de entrada ===
entries = ((df_test['signal'] == 1) & (df_test['pred'] == 1)).fillna(False).astype(bool)

# === 6. Señales de salida ===
tp_pips = 0.00050
horizon = 48
exits = pd.Series(False, index=df_test.index)

for i in entries[entries].index:
    entry_price = df_test.at[i, 'close']
    future = df_test.iloc[i+1:min(i+1+horizon, len(df_test))]

    if not future.empty and (future['high'] >= entry_price + tp_pips).any():
        tp_hit_idx = future[future['high'] >= entry_price + tp_pips].index[0]
        exits.at[tp_hit_idx] = True
    elif not future.empty:
        fallback_idx = future.index[-1]
        exits.at[fallback_idx] = True

# === 7. Preparar precios ===
prices = df_test['close'].astype(float).ffill().bfill()
entries = entries.reindex(prices.index).fillna(False).astype(bool)
exits = exits.reindex(prices.index).fillna(False).astype(bool)

# === 8. Configuración de trading ===
init_cash = 10000
leverage = 30
lot_size = 100
fixed_size = float(lot_size * leverage)

# === 9. Simulación con VectorBT ===
portfolio = vbt.Portfolio.from_signals(
    close=prices,
    entries=entries,
    exits=exits,
    size=fixed_size,
    init_cash=init_cash,
    fees=0.001,
    slippage=0.0005,
    direction='longonly'
)

# === 10. Estadísticas del portafolio ===
print("\n📊 Estadísticas del portafolio (modelo Keras):")
stats = portfolio.stats()
print(stats)


✅ Modelo cargado: best_model_walkforward_keras_optuna.h5
194/194 ━━━━━━━━━━━━━━━━━━━━ 0s 721us/step

📊 Estadísticas del portafolio (modelo Keras):
Start                             0.000000
End                            6191.000000
Period                         6192.000000
Start Value                   10000.000000
End Value                      9367.695020
Total Return [%]                 -6.323050
Benchmark Return [%]             -9.224518
Max Gross Exposure [%]           28.947087
Total Fees Paid                 374.436665
Max Drawdown [%]                  6.352177
Max Drawdown Duration          5947.000000
Total Trades                     71.000000
Total Closed Trades              71.000000
Total Open Trades                 0.000000
Open Trade PnL                    0.000000
Win Rate [%]                     14.084507
Best Trade [%]                    0.795829
Worst Trade [%]                  -1.531093
Avg Winning Trade [%]             0.237324
Avg Losing Trade [%]             -0.

c:\Users\sebas\OneDrive\Escritorio\TFM-Ciencia de datos\venv\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning: Metric 'sharpe_ratio' requires frequency to be set
  warnings.warn(warning_message)
c:\Users\sebas\OneDrive\Escritorio\TFM-Ciencia de datos\venv\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning: Metric 'calmar_ratio' requires frequency to be set
  warnings.warn(warning_message)
c:\Users\sebas\OneDrive\Escritorio\TFM-Ciencia de datos\venv\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning: Metric 'omega_ratio' requires frequency to be set
  warnings.warn(warning_message)
c:\Users\sebas\OneDrive\Escritorio\TFM-Ciencia de datos\venv\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning: Metric 'sortino_ratio' requires frequency to be set
  warnings.warn(warning_message)


In [2]:
import os
os.environ["NUMBA_DISABLE_JIT"] = "1"

import pandas as pd
import numpy as np

# === 1. Cargar df_test ===
df_test = pd.read_csv("df_test.csv").reset_index(drop=True)

if 'time' in df_test.columns:
    df_test['time'] = pd.to_datetime(df_test['time'])
    df_test['hour'] = df_test['time'].dt.hour
    df_test['weekday'] = df_test['time'].dt.weekday

# === 2. Entradas basadas en señal Renko/MRB sin modelo ===
entries = (df_test['signal'] == 1).fillna(False).astype(bool)

# === 3. Salidas basadas en take-profit o timeout ===
tp_pips = 0.00050
horizon = 48
exits = pd.Series(False, index=df_test.index)

for i in entries[entries].index:
    entry_price = df_test.at[i, 'close']
    future = df_test.iloc[i+1:min(i+1+horizon, len(df_test))]

    if not future.empty and (future['high'] >= entry_price + tp_pips).any():
        tp_hit_idx = future[future['high'] >= entry_price + tp_pips].index[0]
        exits.at[tp_hit_idx] = True
    elif not future.empty:
        fallback_idx = future.index[-1]
        exits.at[fallback_idx] = True

# === 4. Preprocesamiento final ===
prices = df_test['close'].astype(float).ffill().bfill()
entries = entries.reindex(prices.index).fillna(False).astype(bool)
exits = exits.reindex(prices.index).fillna(False).astype(bool)

# === 5. Configuración de trading ===
init_cash = 10000
leverage = 30
lot_size = 100
fixed_size = float(lot_size * leverage)

# === 6. Simulación del equity curve manual ===
capital = init_cash
positions = []
equity_curve = []
total_trades = win_trades = loss_trades = total_fees = 0

entry_dates = []
exit_dates = []
profits = []

for i in range(len(prices)):
    if entries[i]:
        entry_price = prices[i]
        entry_dates.append(df_test.at[i, 'time'] if 'time' in df_test.columns else pd.NaT)
        positions.append((entry_price, i))
    elif exits[i] and positions:
        entry_price, entry_idx = positions.pop()
        exit_price = prices[i]
        trade_pnl = (exit_price - entry_price) * fixed_size
        capital += trade_pnl
        fee = abs(trade_pnl) * 0.001
        total_fees += fee
        profits.append(trade_pnl)
        exit_dates.append(df_test.at[i, 'time'] if 'time' in df_test.columns else pd.NaT)

        total_trades += 1
        if trade_pnl > 0:
            win_trades += 1
        else:
            loss_trades += 1

    equity_curve.append(capital)

# === 7. Estadísticas ===
total_return = (capital - init_cash) / init_cash * 100
equity_curve = np.array(equity_curve)
peak = np.maximum.accumulate(equity_curve)
drawdown = (equity_curve - peak) / peak * 100
max_drawdown = drawdown.min()
win_rate = (win_trades / total_trades) * 100 if total_trades else 0
avg_win = np.mean([p for p in profits if p > 0]) if win_trades else 0
avg_loss = np.mean([p for p in profits if p < 0]) if loss_trades else 0
profit_factor = -sum([p for p in profits if p > 0]) / sum([p for p in profits if p < 0]) if loss_trades else np.inf
expectancy = (win_rate / 100) * avg_win + ((100 - win_rate) / 100) * avg_loss

# Duraciones
winning_durations = []
losing_durations = []
for start, end, pnl in zip(entry_dates, exit_dates, profits):
    if pd.notna(start) and pd.notna(end):
        duration = (end - start).days
        if pnl > 0:
            winning_durations.append(duration)
        else:
            losing_durations.append(duration)

avg_win_dur = np.mean(winning_durations) if winning_durations else 0
avg_loss_dur = np.mean(losing_durations) if losing_durations else 0

# === 8. Mostrar resultados ===
print("\n📊 Backtest sin modelo (Renko/MRB):")
print(f"Capital final: {capital:.2f}")
print(f"Ganancia/Pérdida neta: {capital - init_cash:.2f}")
print(f"Retorno total [%]: {total_return:.2f}")
print(f"Max Drawdown [%]: {max_drawdown:.2f}")
print(f"Total Fees Paid: {total_fees:.2f}")
print(f"Total Trades: {total_trades}")
print(f"Win Rate [%]: {win_rate:.2f}")
print(f"Avg Win Trade: {avg_win:.2f}")
print(f"Avg Loss Trade: {avg_loss:.2f}")
print(f"Avg Win Duration: {avg_win_dur:.2f} days")
print(f"Avg Loss Duration: {avg_loss_dur:.2f} days")
print(f"Profit Factor: {profit_factor:.2f}")
print(f"Expectancy: {expectancy:.2f}")



📊 Backtest sin modelo (Renko/MRB):
Capital final: 9067.60
Ganancia/Pérdida neta: -932.40
Retorno total [%]: -9.32
Max Drawdown [%]: -9.43
Total Fees Paid: 1.32
Total Trades: 159
Win Rate [%]: 48.43
Avg Win Trade: 2.52
Avg Loss Trade: -13.74
Avg Win Duration: 143.61 days
Avg Loss Duration: 135.28 days
Profit Factor: 0.17
Expectancy: -5.86


In [3]:
import os
os.environ["NUMBA_DISABLE_JIT"] = "1"

import pandas as pd
import numpy as np
import vectorbt as vbt
import ta
from tensorflow.keras.models import load_model

# === 1. Cargar df_test original y calcular estadísticos base ===
df_base = pd.read_csv("df_test.csv")
df_base['time'] = pd.to_datetime(df_base['time'])

n_rows = len(df_base)
np.random.seed(42)

# === 2. Simulación Monte Carlo de precios OHLC ===
close_sim = np.cumsum(np.random.normal(0, 0.0008, n_rows)) + df_base['close'].iloc[0]
open_sim = close_sim + np.random.normal(0, 0.0003, n_rows)
high_sim = np.maximum(open_sim, close_sim) + np.abs(np.random.normal(0, 0.0005, n_rows))
low_sim = np.minimum(open_sim, close_sim) - np.abs(np.random.normal(0, 0.0005, n_rows))
volume_sim = np.abs(np.random.normal(1000, 300, n_rows))

df_sim = pd.DataFrame({
    'time': pd.date_range(start='2023-01-01', periods=n_rows, freq='H'),
    'open': open_sim,
    'high': high_sim,
    'low': low_sim,
    'close': close_sim,
    'tick_volume': volume_sim
})

df_sim['hour'] = df_sim['time'].dt.hour
df_sim['weekday'] = df_sim['time'].dt.weekday

# === 3. Calcular features técnicas ===
df_sim['sma_200_vs_100'] = ta.trend.sma_indicator(df_sim['close'], 200) - ta.trend.sma_indicator(df_sim['close'], 100)
df_sim['sma_50_vs_25'] = ta.trend.sma_indicator(df_sim['close'], 50) - ta.trend.sma_indicator(df_sim['close'], 25)
df_sim['rsi_14'] = ta.momentum.rsi(df_sim['close'], window=14)
df_sim['acceleration_12'] = df_sim['close'].diff().diff(periods=12)
df_sim['macd'] = ta.trend.macd_diff(df_sim['close'])
df_sim['stoch_k'] = ta.momentum.stoch(df_sim['high'], df_sim['low'], df_sim['close'])
df_sim['momentum_12'] = df_sim['close'].diff(periods=12)
bb = ta.volatility.BollingerBands(df_sim['close'], window=20)
df_sim['bb_middle'] = bb.bollinger_mavg()
df_sim['bb_upper'] = bb.bollinger_hband()
df_sim['bb_lower'] = bb.bollinger_lband()
df_sim['weighted_close'] = (df_sim['high'] + df_sim['low'] + 2 * df_sim['close']) / 4
df_sim['poc_12'] = df_sim['close'].rolling(12).mean()
df_sim['high_channel_20'] = df_sim['high'].rolling(20).max()
df_sim['low_channel_20'] = df_sim['low'].rolling(20).min()
df_sim['abs_diff_mrb_vs_close'] = np.abs(df_sim['close'] - df_sim['close'].rolling(20).mean())

for i in range(1, 6):
    df_sim[f'last_signal_price_{i}'] = df_sim['close'].shift(i)

df_sim['signal'] = np.random.choice([0, 1], size=n_rows, p=[0.8, 0.2])

# === 4. Cargar modelo Keras ===
features_cols = [
    'close', 'sma_200_vs_100', 'sma_50_vs_25', 'rsi_14', 'acceleration_12',
    'macd', 'stoch_k', 'momentum_12', 'bb_middle', 'bb_upper',
    'bb_lower', 'weighted_close', 'poc_12', 'high_channel_20',
    'low_channel_20', 'abs_diff_mrb_vs_close',
    'last_signal_price_1', 'last_signal_price_2',
    'last_signal_price_3', 'last_signal_price_4',
    'last_signal_price_5', 'hour', 'weekday'
]

model = load_model("best_model_walkforward_keras_optuna.h5")

X_sim = df_sim[features_cols].fillna(0).values
proba_1 = model.predict(X_sim).flatten()
df_sim['proba_1'] = proba_1
df_sim['pred'] = (proba_1 > 0.5).astype(int)

# === 5. Señales de entrada/salida ===
entries = ((df_sim['signal'] == 1) & (df_sim['pred'] == 1)).fillna(False)
tp_pips = 0.0012
horizon = 20
exits = pd.Series(False, index=df_sim.index)

for i in entries[entries].index:
    entry_price = df_sim.at[i, 'close']
    future = df_sim.iloc[i+1:min(i+1+horizon, len(df_sim))]

    if not future.empty and (future['high'] >= entry_price + tp_pips).any():
        tp_hit_idx = future[future['high'] >= entry_price + tp_pips].index[0]
        exits.at[tp_hit_idx] = True
    elif not future.empty:
        exits.at[future.index[-1]] = True

# === 6. Simulación VectorBT ===
prices = df_sim['close'].astype(float)
entries = entries.reindex(prices.index).fillna(False).astype(bool)
exits = exits.reindex(prices.index).fillna(False).astype(bool)

init_cash = 10000
leverage = 30
lot_size = 100
fixed_size = lot_size * leverage

portfolio = vbt.Portfolio.from_signals(
    close=prices,
    entries=entries,
    exits=exits,
    size=fixed_size,
    init_cash=init_cash,
    fees=0.001,
    slippage=0.0005,
    direction='longonly'
)

# === 7. Resultados ===
print("\n📊 Simulación con Monte Carlo usando modelo Keras:")
stats = portfolio.stats()

# Imprimir resumen
for key in [
    "Start Value", "End Value", "Total Return [%]", "Benchmark Return [%]",
    "Max Gross Exposure [%]", "Total Fees Paid", "Max Drawdown [%]",
    "Max Drawdown Duration", "Total Trades", "Total Closed Trades",
    "Total Open Trades", "Open Trade PnL", "Win Rate [%]", "Best Trade [%]",
    "Worst Trade [%]", "Avg Winning Trade [%]", "Avg Losing Trade [%]",
    "Avg Winning Trade Duration", "Avg Losing Trade Duration",
    "Profit Factor", "Expectancy"
]:
    print(f"{key}: {stats[key]}")


C:\Users\sebas\AppData\Local\Temp\ipykernel_22456\2308999762.py:25: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  'time': pd.date_range(start='2023-01-01', periods=n_rows, freq='H'),


194/194 ━━━━━━━━━━━━━━━━━━━━ 0s 745us/step

📊 Simulación con Monte Carlo usando modelo Keras:


c:\Users\sebas\OneDrive\Escritorio\TFM-Ciencia de datos\venv\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning: Metric 'sharpe_ratio' requires frequency to be set
  warnings.warn(warning_message)
c:\Users\sebas\OneDrive\Escritorio\TFM-Ciencia de datos\venv\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning: Metric 'calmar_ratio' requires frequency to be set
  warnings.warn(warning_message)
c:\Users\sebas\OneDrive\Escritorio\TFM-Ciencia de datos\venv\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning: Metric 'omega_ratio' requires frequency to be set
  warnings.warn(warning_message)
c:\Users\sebas\OneDrive\Escritorio\TFM-Ciencia de datos\venv\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning: Metric 'sortino_ratio' requires frequency to be set
  warnings.warn(warning_message)


Start Value: 10000.0
End Value: 5673.342625668976
Total Return [%]: -43.26657374331024
Benchmark Return [%]: -0.7550245020410129
Max Gross Exposure [%]: 47.43485425573051
Total Fees Paid: 3079.7877889945566
Max Drawdown [%]: 43.266573743310396
Max Drawdown Duration: 6168.0
Total Trades: 544.0
Total Closed Trades: 544.0
Total Open Trades: 0.0
Open Trade PnL: 0.0
Win Rate [%]: 1.2867647058823528
Best Trade [%]: 0.18528876571140615
Worst Trade [%]: -1.2068712870287035
Avg Winning Trade [%]: 0.06639201228866716
Avg Losing Trade [%]: -0.28548309066075
Avg Winning Trade Duration: 8.857142857142858
Avg Losing Trade Duration: 5.867783985102421
Profit Factor: 0.002957730997495421
Expectancy: -7.953414291049661
